In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

warnings.filterwarnings("ignore")

In [2]:
def sort_reset_date(df, col_d='WORK_DATE'):
    """ Sort and reset dataframes based on the date and time

    Args:
        df: dataframes containing column about the date
        col_d: date column of the dataframe, to be manipulated

    Returns:
        df: sorted dataframe with reset index
    """
    
    df.sort_values(by=col_d, inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    return df

In [3]:
def hour_rounder(df, col_t='DEB_TIME'):

    df['TIME_HOUR'] = pd.to_datetime(df[col_t]).dt.floor('60min')
    
    return df

In [4]:
attendance = pd.read_csv(r'data/attendance.csv')
entity_schedule = pd.read_csv(r'data/entity_schedule.csv')
# glossary = pd.read_excel(r'data/glossary.xlsx')
link_attraction = pd.read_csv(r'data/link_attraction_park.csv', sep=';')
parade_night_show = pd.read_excel(r'data/parade_night_show.xlsx')
waiting_times = pd.read_csv(r'data/waiting_times.csv')
weather_data = pd.read_csv(r'data/weather_data.csv')

parade_night_show.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
attendance = attendance[attendance['FACILITY_NAME'] == 'PortAventura World']
lst_attr = link_attraction[link_attraction['PARK'] == 'PortAventura World']['ATTRACTION']
entity_schedule_pa = entity_schedule[entity_schedule['ENTITY_DESCRIPTION_SHORT'].isin(lst_attr)]
waiting_times = waiting_times[waiting_times['ENTITY_DESCRIPTION_SHORT'].isin(lst_attr)]
attendance = attendance[attendance['attendance']>=0]

In [6]:
# Sort Dates and reset index
attendance = sort_reset_date(attendance, 'USAGE_DATE')
entity_schedule_pa = sort_reset_date(entity_schedule_pa, 'DEB_TIME')
parade_night_show = sort_reset_date(parade_night_show, 'WORK_DATE')
waiting_times = sort_reset_date(waiting_times, 'DEB_TIME')
weather_data = sort_reset_date(weather_data, 'dt_iso')

# Delete columns and some preprocessing
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'].str.replace(" +0000 UTC", "", regex=False), errors='coerce', format='%Y-%m-%d %H:%M:%S')
cols_del = ['city_name', 'lat', 'lon', 'weather_id', 'visibility', 'sea_level', 'grnd_level', 'wind_gust', 'snow_3h']
weather_data.drop(columns=cols_del, inplace=True)
weather_data.fillna(0, inplace=True)
attendance.rename(columns={'USAGE_DATE': 'WORK_DATE'}, inplace=True)
# weather_data.rename(columns={'dt_iso': 'TIME_HOUR'}, inplace=True)

attendance['WORK_DATE'] = pd.to_datetime(attendance['WORK_DATE'])
waiting_times['WORK_DATE'] = pd.to_datetime(waiting_times['WORK_DATE'])
parade_night_show['WORK_DATE'] = pd.to_datetime(parade_night_show['WORK_DATE'])

weather_data = hour_rounder(weather_data, col_t='dt_iso')
waiting_times = hour_rounder(waiting_times, col_t='DEB_TIME')
entity_schedule_pa = hour_rounder(entity_schedule_pa, col_t='DEB_TIME')

In [7]:
# Merge the dataframes
df = waiting_times.merge(attendance[['WORK_DATE', 'attendance']], how='left', on='WORK_DATE')
df = df.merge(entity_schedule_pa[['TIME_HOUR', 'ENTITY_DESCRIPTION_SHORT', 'REF_CLOSING_DESCRIPTION', 'ENTITY_TYPE', 'UPDATE_TIME']], how='left', on=['TIME_HOUR', 'ENTITY_DESCRIPTION_SHORT'])
df = df.merge(parade_night_show, how='left', on='WORK_DATE')
df = df.merge(weather_data, how='left', on='TIME_HOUR')

In [8]:
df

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,...,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,clouds_all,weather_main,weather_description,weather_icon
0,2018-01-01,2018-01-01 09:00:00.000,9,2018-01-01 09:15:00.000,Rapids Ride,0,2.0,0.0,0.000,0.00,...,83,7.48,196,0.0,0.0,0.0,100,Clouds,overcast clouds,04d
1,2018-01-01,2018-01-01 09:00:00.000,9,2018-01-01 09:15:00.000,Merry Go Round,0,65.0,0.0,0.000,0.00,...,83,7.48,196,0.0,0.0,0.0,100,Clouds,overcast clouds,04d
2,2018-01-01,2018-01-01 09:00:00.000,9,2018-01-01 09:15:00.000,Bumper Cars,5,18.0,108.0,254.749,254.75,...,83,7.48,196,0.0,0.0,0.0,100,Clouds,overcast clouds,04d
3,2018-01-01,2018-01-01 09:00:00.000,9,2018-01-01 09:15:00.000,Go-Karts,0,2.0,0.0,0.000,0.00,...,83,7.48,196,0.0,0.0,0.0,100,Clouds,overcast clouds,04d
4,2018-01-01,2018-01-01 09:00:00.000,9,2018-01-01 09:15:00.000,Spinning Coaster,0,1.4,0.0,315.750,52.65,...,83,7.48,196,0.0,0.0,0.0,100,Clouds,overcast clouds,04d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369816,2022-08-18,2022-08-18 22:45:00.000,22,2022-08-18 23:00:00.000,Free Fall,0,1.0,0.0,0.000,0.00,...,84,4.20,348,0.0,0.0,0.0,100,Clouds,overcast clouds,04n
2369817,2022-08-18,2022-08-18 22:45:00.000,22,2022-08-18 23:00:00.000,Giant Wheel,10,5.0,252.0,503.750,503.75,...,84,4.20,348,0.0,0.0,0.0,100,Clouds,overcast clouds,04n
2369818,2022-08-18,2022-08-18 22:45:00.000,22,2022-08-18 23:00:00.000,Rapids Ride,0,2.0,0.0,0.000,0.00,...,84,4.20,348,0.0,0.0,0.0,100,Clouds,overcast clouds,04n
2369819,2022-08-18,2022-08-18 22:45:00.000,22,2022-08-18 23:00:00.000,Haunted House,0,9.0,0.0,0.000,0.00,...,84,4.20,348,0.0,0.0,0.0,100,Clouds,overcast clouds,04n
